In [ ]:
class GradientDescentClassifier:
    def __init__(self, learning_rate=0.01, lambda1=0.1, lambda2=0.1, epochs=1000):
        self.learning_rate = learning_rate
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.epochs = epochs
        self.coef_ = None

    def get_params(self, deep=True):
        return {
            'learning_rate': self.learning_rate,
            'lambda1': self.lambda1,
            'lambda2': self.lambda2,
            'epochs': self.epochs
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def hinge_loss(self, margin):
        return np.maximum(0, 1 - margin)

    def perceptron_loss(self, margin):
        return np.maximum(0, -margin)

    def logistic_loss(self, margin):
        return np.log(1 + np.exp(-margin))

    def fit(self, X, y, loss_type="hinge"):
        # Преобразуем X в плотную матрицу
        if hasattr(X, 'toarray'):
            X = X.toarray()

        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)

        # Преобразуем y в массив и задаем его размерность
        y = np.array(y).reshape(-1)

        loss_functions = {
            "hinge": self.hinge_loss,
            "perceptron": self.perceptron_loss,
            "logistic": self.logistic_loss
        }
        loss_func = loss_functions[loss_type]

        for epoch in range(self.epochs):
            margins = y * (X @ self.coef_)
            loss = loss_func(margins)

            grad = -np.mean((X.T * y * (loss > 0)).T, axis=0) + self.lambda1 * np.sign(
                self.coef_) + self.lambda2 * self.coef_

            self.coef_ -= self.learning_rate * grad

    def predict(self, X):
        return np.sign(X @ self.coef_)

In [1]:
class RidgeRegression:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        # Добавляем столбец единиц для учета свободного члена
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        # Формула для Ridge регрессии: (X^T X + alpha * I)^(-1) X^T y
        I = np.eye(X.shape[1])
        I[0, 0] = 0  # не регуляризируем свободный член
        self.coef_ = np.linalg.inv(X.T @ X + self.alpha * I) @ X.T @ y

    def predict(self, X):
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        return X @ self.coef_

In [ ]:
import numpy as np

class SVM:
    def __init__(self, C=1.0, kernel="linear", gamma=1, degree=3):
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.alphas = None
        self.b = 0
        self.support_vectors = None

    def _linear_kernel(self, x1, x2):
        return np.dot(x1, x2)

    def _polynomial_kernel(self, x1, x2):
        return (1 + np.dot(x1, x2)) ** self.degree

    def _rbf_kernel(self, x1, x2):
        return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)

    def _kernel(self, x1, x2):
        if self.kernel == "linear":
            return self._linear_kernel(x1, x2)
        elif self.kernel == "polynomial":
            return self._polynomial_kernel(x1, x2)
        elif self.kernel == "rbf":
            return self._rbf_kernel(x1, x2)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.alphas = np.zeros(n_samples)
        self.b = 0

        def compute_L_H(alpha_i, alpha_j, y_i, y_j):
            if y_i != y_j:
                return (max(0, alpha_j - alpha_i), min(self.C, self.C + alpha_j - alpha_i))
            else:
                return (max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j))

    def predict(self, X):
        return np.sign(np.array([np.sum(self.alphas * self.support_vectors[:, i]) for i in range(len(X))]))